

Assistente Virtual

Utilizando as bibliotecas de Python, iremos criar uma assistente virtual `comandada por voz` que seja capaz de realizar 3 comandos distintos:

    1) A assistente deve receber informações de compromissos/lembretes e **registrá-los** numa "agenda" (para facilitar a "agenda" pode ser um arquivo TXT ou CSV, como preferirem)
    2) A assistente deve informar os compromissos do dia (ex: ler o arquivo e falar os compromissos de uma data)
    3) Crie um terceiro comando livre. Pode ser abrir um arquivo de música, programar um alarme ou timer, iniciar um programa qualquer do seu computador, fazer uma busca na web, retornar o tempo (clima) atual, etc...


#Pipeline

Após executar todos os blocos de install e import, deverá ser executada todas as funções do programa, como gravar áudio, falar texto, retornar o compromisso, etc. Então, a execução principal começa com a assistente contendo comandos de Bom dia, Editar e verificar a agenda e ver o clima completo.

#Installs/Imports


##Dependencias e funções para STT

In [ ]:
!pip install ffmpeg-python
!pip install SpeechRecognition
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
#!pip install PyAudio
#!pip -q install pydub

     |████████████████████████████████| 32.8 MB 38 kB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 1s (189 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 155013 

In [ ]:
# Códigos prontos para solicitar permissão de uso do microfone do computador pelo browser

"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
def mic2wav(audio, wav_file):
  scaled = np.int16(audio/np.max(np.abs(audio)) * 32767)
  write(wav_file, 44100, scaled)

In [ ]:
def transcreve_audio(wav_file):

  # Instancia o reconhecedor de voz
  recon = speechRecog.Recognizer()

  # Converte audio (.wav) para AudioFile
  audiofile = speechRecog.AudioFile(wav_file)

  # Converte AudioFile para AudioData
  with audiofile as source:
      audiodata = recon.record(audiofile)

  # Transcreve AudioData para texto
  text = recon.recognize_google(audiodata, language="pt")

  return text

##Bibiliotecas

In [ ]:
!pip install gtts #Google Text to Speech
!pip install translators --upgrade

     |████████████████████████████████| 6.3 MB 8.1 MB/s 
     |████████████████████████████████| 62 kB 804 kB/s 
     |████████████████████████████████| 57 kB 4.9 MB/s 
     |████████████████████████████████| 81 kB 10.8 MB/s 
     |████████████████████████████████| 65 kB 3.6 MB/s 
  Created wheel for PyExecJS: filename=PyExecJS-1.5.1-py3-none-any.whl size=14596 sha256=04fba37aa1bc1136842f6edcca032ba5f6bd87a988a87e8503106c0cc27cffc7
  Stored in directory: /root/.cache/pip/wheels/9a/ee/03/da5c0b4a8c13362beeb844eb913bbe58a89bde1de2b9157007
Successfully built PyExecJS
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

In [ ]:
from gtts import gTTS #Text to Speech
from IPython.display import Audio #Player de audio
import requests, json #Trabalhar com api de clima
from datetime import date #Trabalhar com datas
from datetime import datetime #Trabalhar com datas

import pandas as pd #Trabalhar com dataframe


import time
from IPython.display import clear_output
import time
import numpy as np
from scipy.io.wavfile import write
import speech_recognition as speechRecog
import translators as ts

Using United States server backend.


#Funções

##Trabalhando com CSV da agenda

In [ ]:
#Lendo dataset
dc1 = pd.read_csv("datas_compromisso.csv")
dc1.head(5)

,Data,Compromisso
0,26/09/2021,Compras do mês
1,27/09/2021,NaN
2,28/09/2021,NaN
3,29/09/2021,Trabalho da faculdade
4,30/09/2021,NaN


In [ ]:
#Substituindo valores nulos para "Nenhum compromisso para hoje"
dc1["Compromisso"] = dc1["Compromisso"].fillna("Nenhum compromisso para hoje")

In [ ]:
dc1.head(10)

,Data,Compromisso
0,26/09/2021,Compras do mês
1,27/09/2021,Nenhum compromisso para hoje
2,28/09/2021,Nenhum compromisso para hoje
3,29/09/2021,Trabalho da faculdade
4,30/09/2021,Nenhum compromisso para hoje
5,01/10/2021,Nenhum compromisso para hoje
6,02/10/2021,Entregar trabalho
7,03/10/2021,Nenhum compromisso para hoje
8,04/10/2021,Sair com os amigos
9,05/10/2021,Jantar com a familia


##Recolhendo informações do clima

In [ ]:
#Api externa com informações climaticas
#https://openweathermap.org/

def getWeather(tipo):

  api_key = "66cb72f9e399d6dad60e7b2fb8ff7aca"

  base_url = "http://api.openweathermap.org/data/2.5/weather?"

  #city_name = input("Enter city name : ")
  city_name = "São Paulo"

  complete_url = base_url + "appid=" + api_key + "&q=" + city_name

  response = requests.get(complete_url)

  x = response.json()

  if x["cod"] != "404":

      y = x["main"]

      current_temperature = y["temp"] - (273.15)

      current_wind = x["wind"]["speed"] * 3.6

      current_humidity = y["humidity"]

      weather_description = x["weather"][0]["description"]

      weather_description = ts.google(weather_description, from_language="en", to_language="pt")

      if tipo == "temp":
        txt1 = "{temp:.0f}°C".format(temp = current_temperature)
      else:
        txt1 = "Temperatura: {temp:.0f}°C".format(temp = current_temperature)
        txt1 += "\nUmidade: {humi}%".format(humi = current_humidity)
        txt1 += "\nVento: {wind:.0f} km/h".format(wind = current_wind)
        txt1 += "\Previsão: {weather}".format(weather = weather_description).title()

      #print(txt1)
      return(txt1)

  else:
      print(" City Not Found ")

##Sysdate

In [ ]:
#Função para retornar o dia atual
def getDate():

  data_atual = date.today()
  td = timedelta(days=-1)
  data_atual = data_atual + td
  txtDataAtual = "{}/{}/{}".format(f"{data_atual.day:02d}", f"{data_atual.month:02d}",
  data_atual.year)

  return(txtDataAtual)

In [ ]:
getDate()

'28/09/2021'

##Retorna Compromisso

In [ ]:
#Função para retornar o compromisso baseado na data passada como parametro
def getCompromisso(data):

  return(dc1.loc[dc1['Data'] == data].at[(dc1.loc[dc1['Data'] == data].index.values.astype(int)[0]), "Compromisso"])

##Text to Speech Dia atual

In [ ]:
def hoje():
  txtFinal = "Olá, hoje é dia " + getDate() + ". Voce tem os seguintes compromissos hoje: " + getCompromisso(getDate()) + ". Temperatura atual: "+ getWeather("temp")
  falar(txtFinal)
  return(Audio("/content/voz.mp3"))
#voz = gTTS(txtFinal, lang="pt")
#voz.save("voz.mp3")
#Audio("/content/voz.mp3")

##Gravar Audio

In [ ]:
#Função para gravar audio e retornar o texto "STT"
def gravar():
  audio, sr = get_audio()
  mic2wav(audio, "test3.wav")
  texto = transcreve_audio("test3.wav")
  return(texto)

##Falar texto

In [ ]:
#Função com string como parametro e salvar o audio "TTS"
def falar(fala):
  voz = gTTS(fala, lang="pt")
  voz.save("voz.mp3")
  #return(Audio("/content/voz.mp3"))

##Formata Data

In [ ]:
#Formatar a data, exemplo: 29 de setembro de 2021 -> 29/09/2021
def retData(data):

  texto = ts.google(data, from_language="pt", to_language="en")

  texto = datetime.strptime(texto, '%B %d, %Y')

  texto = "{}/{}/{}".format(f"{texto.day:02d}", f"{texto.month:02d}",texto.year)
  texto = str(texto)
  texto = texto.replace(" 00:00:00", "")
  texto = texto.replace("-", "/")

  return(texto)

##Inserir/Editar compromissos

In [ ]:
#Função para alterar compromisso, recebe como parametro as opções (novo,substituir, apagar), data e o compromisso para marcar
def edita(qst, data, compro):

  if (qst == "novo"):

    print(data)
    print(compro)

    if((dc1.at[(dc1.loc[dc1['Data'] == data].index.values.astype(int)[0]), "Compromisso"]) == "Nenhum compromisso para hoje"):
      dc1.at[(dc1.loc[dc1['Data'] == data].index.values.astype(int)[0]), "Compromisso"] = compro.capitalize()
    else:
      dc1.at[(dc1.loc[dc1['Data'] == data].index.values.astype(int)[0]), "Compromisso"] += (", "+compro.capitalize())

    return(falar("Compromisso marcado"))

  elif (qst == "substituir"):

    dc1.at[(dc1.loc[dc1['Data'] == data].index.values.astype(int)[0]), "Compromisso"] = compro.capitalize()
    return(falar("Compromisso marcado"))

  elif (qst == "apagar"):

    dc1.at[(dc1.loc[dc1['Data'] == data].index.values.astype(int)[0]), "Compromisso"] = "Nenhum compromisso para hoje"
    return(falar("Compromisso excluido"))

  else:
    return(falar("Não entendi o que voce disse"))

dc1.to_csv('datas_compromisso.csv', index=False)

##Verificar Dia X

In [ ]:
#Função para retornar o um dia especifico, usando data como parametro
def compromissoDia(data):

  return("Olá, para o dia " + data + ". Voce terá os seguintes compromissos : " + getCompromisso(data))

#voz = gTTS(txtFinal, lang="pt")
#voz.save("voz.mp3")
#Audio("/content/voz.mp3")

#Main

##Assistente

In [ ]:
#Cabeçalho com opções
print("Comandos de voz: \nBom Dia \nVerificar Agenda \nEditar Agenda \nClima \nSair\n")
time.sleep(3)
opcao = gravar()
#print(opcao)

#Seleção das opções

if opcao == "Bom dia" or opcao == "bom dia":
  hoje()

elif opcao == "Verificar agenda" or opcao == "verificar agenda":

  print("\n\nQual data?")

  time.sleep(2)

  data = gravar()
  falar(compromissoDia(retData(data)))

elif opcao == "Editar agenda" or opcao == "editar agenda":
  print("\n\nNovo compromisso, substituir ou apagar compromisso? \nQual data? \nQual compromisso?")

  time.sleep(4)

  qst = gravar()
  data = gravar()

  if(qst == "apagar"):
    edita(qst, retData(data), None)
  else:
    compro = gravar()
    edita(qst, retData(data), compro)

elif opcao == "Clima" or opcao == "clima":
  falar(getWeather("full"))

elif opcao == "Sair" or opcao == "sair":
  falar("Até mais!")

else:
  falar("Opção inválida!")

Audio("/content/voz.mp3")


#Rascunho


In [ ]:
dc1.head(10)

,Data,Compromisso
0,26/09/2021,Compras do mês
1,27/09/2021,Nenhum compromisso para hoje
2,28/09/2021,Nenhum compromisso para hoje
3,29/09/2021,Trabalho da faculdade
4,30/09/2021,Nenhum compromisso para hoje
5,01/10/2021,Nenhum compromisso para hoje
6,02/10/2021,Entregar trabalho
7,03/10/2021,Nenhum compromisso para hoje
8,04/10/2021,Sair com os amigos
9,05/10/2021,Jantar com a familia
